In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
import torch.nn.functional as F

In [3]:
# Load MNIST dataset (subset for faster training)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform)

# Use only a small subset to speed up training
train_subset = Subset(trainset, np.random.choice(len(trainset), 5000, replace=False))
test_subset = Subset(testset, np.random.choice(len(testset), 1000, replace=False))

train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

# Define different models
class CNNModel(nn.Module):  # No hidden layer, Softmax output
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        self.conv_class = nn.Conv2d(64, 10, kernel_size=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = self.conv_class(x)
        x = F.adaptive_avg_pool2d(x, (1,1))
        x = x.view(x.size(0), -1)

        return torch.softmax(x, dim=1)

# 3. Initialize Model and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 4. Training Loop
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# 5. Testing Loop
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)')

# 6. Run Training and Testing
for epoch in range(1, 25):
    train(epoch)
    test()

# 7. Example Inference
def predict(image):
    model.eval()
    with torch.no_grad():
        image = image.to(device).unsqueeze(0)  # Add batch dimension
        output = model(image)
        probabilities = torch.softmax(output, dim=1)
    return probabilities.cpu().numpy().flatten()

100%|██████████| 9.91M/9.91M [00:00<00:00, 56.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.68MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 4.45MB/s]


Test set: Average loss: 0.0362, Accuracy: 168/1000 (17%)
Test set: Average loss: 0.0349, Accuracy: 331/1000 (33%)
Test set: Average loss: 0.0341, Accuracy: 336/1000 (34%)
Test set: Average loss: 0.0335, Accuracy: 371/1000 (37%)
Test set: Average loss: 0.0330, Accuracy: 447/1000 (45%)
Test set: Average loss: 0.0324, Accuracy: 495/1000 (50%)
Test set: Average loss: 0.0318, Accuracy: 533/1000 (53%)
Test set: Average loss: 0.0315, Accuracy: 544/1000 (54%)
Test set: Average loss: 0.0310, Accuracy: 561/1000 (56%)
Test set: Average loss: 0.0305, Accuracy: 606/1000 (61%)
Test set: Average loss: 0.0303, Accuracy: 607/1000 (61%)
Test set: Average loss: 0.0299, Accuracy: 651/1000 (65%)
Test set: Average loss: 0.0297, Accuracy: 632/1000 (63%)
Test set: Average loss: 0.0294, Accuracy: 678/1000 (68%)
Test set: Average loss: 0.0290, Accuracy: 703/1000 (70%)
Test set: Average loss: 0.0289, Accuracy: 694/1000 (69%)
Test set: Average loss: 0.0286, Accuracy: 713/1000 (71%)
Test set: Average loss: 0.0285,